In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kelp_compute

In [3]:
import sympy as sp
sp.init_printing()

In [4]:
import numpy as np

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

from fortran_wrappers.pykelp3d_wrap import pykelp3d_wrap as f90
f90.solve_rte_with_callbacks?

In [6]:
space = sp.var('x, y, z')
x, y, z = space
vec_x = sp.Matrix(space)

angle = sp.var(r'theta, phi')
th, ph = angle
vec_om = sp.Matrix([sp.sin(ph)*sp.cos(th), sp.sin(ph)*sp.sin(th), sp.cos(ph)])

angle_prime = sp.var(r'theta_p, phi_p')
thp, php = angle_prime
vec_omp = sp.Matrix([sp.sin(php)*sp.cos(thp), sp.sin(php)*sp.sin(thp), sp.cos(php)])

In [7]:
def grad(expr, space=space):
    return sp.Matrix([sp.diff(expr, d) for d in space])

In [8]:
def sphere_integral(expr, angle):
    theta, phi = angle
    return sp.integrate(expr*sp.sin(phi), (theta, 0, 2*sp.pi), (phi, 0, sp.pi))

In [9]:
def dot(a, b):
    return sum(a.T * b)

---

In [10]:
def calculate_source(L, b, a, beta, params=()):
    L_om = L(*space, *angle, *params)
    L_omp = L(*space, *angle_prime, *params)
    
    deriv = dot(vec_om, grad(L_om))
    atten = (a(*space)+b)*L_om
    
    scat_integrand = dot(vec_om, grad(L_omp))
    scat = sphere_integral(scat_integrand, angle=angle_prime)
    
    source = deriv + atten - scat
    
    return source

In [11]:
def calculate_bc(L, params=()):
    z = space[-1]
    zmin = 0
    return L(*space, *angle, *params).subs(z, zmin)

In [12]:
b = sp.var('b')
params = sp.var('alpha, gamma')

In [13]:
def uniform_vsf(delta):
    return 1/(4*sp.pi)

In [14]:
# TODO: No upwelling light from bottom
# Also, spatially homogeneous BC
def prod_L(x, y, z, theta, phi, alpha, gamma):
    return (
        (
            (
                (2+sp.sin(2*sp.pi*x/alpha))
                *(2+sp.sin(2*sp.pi*y/alpha))
                *(sp.sin(2*sp.pi*z/gamma))
            ) + sp.exp(-z/gamma)
        )
        *(2+sp.sin(phi))
    )

In [15]:
# TODO: Should be periodic on correct grid
def prod_a(x, y, z):
    return (2+sp.sin(2*sp.pi*x))*(2+sp.sin(2*sp.pi*y))*(1+sp.tanh(z-gamma))

In [16]:
prod_source = calculate_source(prod_L, b, prod_a, uniform_vsf, params)
prod_bc = calculate_bc(prod_L, params)

In [17]:
prod_bc

In [18]:
alpha = 1
gamma = 1
b = 0.5

In [19]:
prod_bc

In [20]:
prod_source_N = np.vectorize(
    sp.lambdify(
        (*space, *angle), 
        prod_source.subs('alpha', alpha).subs('gamma', gamma).subs('b', b),
        modules=("sympy", "numpy")
    )
)

prod_a_N = np.vectorize(
    prod_a
)

prod_bc_N = np.vectorize(
    sp.lambdify(
        angle, 
        prod_bc.subs('alpha', alpha).subs('gamma', gamma)
    )
)


In [ ]:
t = np.linspace(0, 1, 101)
x, y = np.meshgrid(t, t, indexing='ij')
z = prod_source_N(x, y, 0, np.pi/4, np.pi/4)

plt.contourf(x, y, z)

In [ ]:
abs_func = prod_a_N
source_func = prod_source_N
bc_func = prod_bc_N
vsf_func = uniform_vsf

In [ ]:
abs_func(1,2,3)

In [ ]:
a = source_func(1,2,3,4,5)

In [ ]:
vsf_func(1)

In [ ]:
bc_func(1,2)

In [ ]:
ns = 6
nz = 6
na = 6
rope_spacing = 1
zmax = 1

lis_opts = "-i gmres"
fd_flag = False
num_scatters = 0

kelp_compute.solve_rte_with_callbacks_full(
    ns, nz, na,
    rope_spacing, zmax,
    b, abs_func, source_func, bc_func, vsf_func,
    num_scatters, fd_flag, lis_opts
)

In [ ]:
abs_func(*space)

In [ ]:
source_func(*space, *angle)